# Analysis


## Parameters

In [ ]:
MEMORY_BACKEND = "mem0" # "memo" or "graphiti"
RESET_MEMORY_BACKENDS = True
NETWORK_PROFILE = "constrained" # "constrained" or "unconstrained"

CONVERSATION_INDEX = 0
NO_OF_SESSIONS = 1

In [ ]:
NETWORK_PROFILE_UNCONSTRAINED = {
    "TOXIC_LATENCY": 0,
    "TOXIC_JITTER": 0,
    "TOXIC_BANDWIDTH": 0,
    "TOXIC_SLOW_CLOSE": 0,
    "TOXIC_TIMEOUT": 0,
    "TOXIC_SLICER": 0,
    "TOXIC_LIMIT_DATA": 0,
    "TOXIC_RESET_PEER": 0,
}

NETWORK_PROFILE_CONSTRAINED = {
    "TOXIC_LATENCY": 200, # ms
    "TOXIC_JITTER": 50, # ms
    "TOXIC_BANDWIDTH": 1000, # KB/s ≈ 1MB/s ≈ 8Mbps
    "TOXIC_SLOW_CLOSE": 0,
    "TOXIC_TIMEOUT": 0,
    "TOXIC_SLICER": 0,
    "TOXIC_LIMIT_DATA": 0,
    "TOXIC_RESET_PEER": 0,
}

if NETWORK_PROFILE == "constrained":
    PROFILE = NETWORK_PROFILE_CONSTRAINED
else:
    PROFILE = NETWORK_PROFILE_UNCONSTRAINED

## Dependencies

In [ ]:
%pip install sentence_transformers

## Reset


In [ ]:
!make -C ../ down

In [ ]:
if RESET_MEMORY_BACKENDS:
    import subprocess
    subprocess.run(["make", "-C", "../", "reset"], check=True)

## Build

In [ ]:
from utils import set_env_var

set_env_var('MEMORY_BACKEND', MEMORY_BACKEND)

In [ ]:
!make -C ../ build-monitoring

In [ ]:
!make -C ../ build-proxy

In [ ]:
!make -C ../ build-dmas

## Run


In [ ]:
!make -C ../ up

In [ ]:
from utils import verify_memory_backend

verify_memory_backend()

## Toxiproxy

In [ ]:
import os

TOXIPROXY_URL = os.environ.get("TOXIPROXY_URL")
TOXIPROXY_URL

In [ ]:
from utils import apply_network_profile

apply_network_profile(PROFILE)

In [ ]:
from utils import check_toxics

check_toxics(PROFILE, TOXIPROXY_URL)

## Memory


Optional: run `docker logs memory -f`


### Load


In [ ]:
from utils import load_memories

memories = load_memories(NO_OF_SESSIONS, CONVERSATION_INDEX, MEMORY_BACKEND)

In [ ]:
from utils import export_loaded_memory_metrics

export_loaded_memory_metrics(memories, MEMORY_BACKEND, CONVERSATION_INDEX)

### Ask


In [ ]:
import requests
import os

LOCOMO_URL = os.getenv("LOCOMO_URL")
questions = requests.get(f"{LOCOMO_URL}/conversations/index/{CONVERSATION_INDEX}/questions").json()
print(questions)

In [ ]:
import requests

COORDINATOR_URL = os.getenv("COORDINATOR_URL")

for question in questions["questions"][:5]:
    print(f"Question: {question['question']}")
    print(f"\nOriginal Answer: {question['answer']}")
    
    # E2E Test
    try:
        resp = requests.post(f"{COORDINATOR_URL}/ask", params={"question": question['question']}, timeout=60)
        new_answer = resp.json().get("answer", "N/A")
        print(f"\nE2E Answer NOW: {new_answer}")
    except Exception as e:
        print(f"\nE2E Answer NOW: ERROR - {e}")
    
    print("=" * 80)

In [ ]:
from utils import run_qa

qa = run_qa(questions, MEMORY_BACKEND, CONVERSATION_INDEX, PROFILE)

In [ ]:
from utils import export_qa

export_qa(qa, MEMORY_BACKEND, CONVERSATION_INDEX)